# Assignment 5: Agent



In this assignment, you will write a program to solve Wordle using LLM agent.

## What is Wordle

Wordle is a word puzzle game where players have **six chances** to guess a hidden **five-letter word**. After each guess, the game provides feedback to help players refine their guesses:

* A **green** letter is in the correct position.
* A **yellow** letter is in the word but in the wrong position.
* A **gray** letter is not in the word at all.

The challenge lies in using these clues strategically to solve the word in the **fewest number of guesses** possible.

You can try Wordle game here:
* https://www.nytimes.com/games/wordle/index.html: The New York Times Wordle (Daily)
* https://wordly.org/

## Environment Setup

In [ ]:
# Install langchain
!pip install -qU langchain

In [ ]:
# Install langchain-openai
!pip install -qU langchain-openai

In [ ]:
!pip install -qU wordle-python

### Set API key

In [ ]:
# Set API key
OPENAI_API_KEY="your_api_key_here"

## Test Wordle

In [ ]:
import wordle

word = "EAGLE"

game = wordle.Wordle(word, real_words = True)

### Validating user guess
response = game.send_guess("WHEAT")
print(response)

In [ ]:
guess = "eater"
response = game.send_guess(guess)
print(response)

In [ ]:
guess = "prompt"
response = game.send_guess(guess)
print(response)

In [ ]:
guess = "abcde"
print(game.send_guess(guess))

In [ ]:
guess = "A bird"
print(game.send_guess(guess))

In [ ]:
guess = "break"
print(game.send_guess(guess))

In [ ]:
guess = "brake"
print(game.send_guess(guess))

In [ ]:
guess = "crazy"
print(game.send_guess(guess))

In [ ]:
guess = "fable"
print(game.send_guess(guess))

In [ ]:
word = "GAMES"

game = wordle.Wordle(word, real_words = True)

### Validating user guess
response = game.send_guess("TREES")
print(response)

In [ ]:
# prompt: check if the response is the tuple or not

if isinstance(response, tuple):
  print("The response is a tuple.")
else:
  print("The response is not a tuple.")

## Define Class for Structured Output

In [ ]:
from pydantic import BaseModel, Field

# TypedDict
class Guess(BaseModel):
    """Wordle guess"""

    reason: str = Field(description = "reason for the guess")
    guess: str = Field(description = "next best word to guess")

## Using Language Models

In [ ]:
# Prepare model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.00, api_key=OPENAI_API_KEY, max_tokens=512)
struct_llm = llm.with_structured_output(Guess)

In [ ]:
from langchain_core.prompts import PromptTemplate

## Test GPT-4o-mini with a simple Agent

### Define Prompt Templates

In [ ]:
guess_prompt = PromptTemplate(
    input_variables=["game_status"],
    template="""
    You are a Wordle solver. Your goal is to guess the secret five-letter word in as few tries as possible.
    You will receive the current game status, including previous guesses and results.
    - Italic Capital Letter (ex: *A*): The letter is in the correct position.
    - Capital Letter (ex: A): The letter is in the word but in the wrong position.
    - Lower Letter (ex: a): The letter is not in the word.
    Based on this information, suggest the next best word to guess.
    Respond in JSON with `guess` and `reason` keys"

    <Example>
    Game Status:
    Game Start!
    Result 1: 'c   *R*   A   t   E   '
    Output JSON:
    {{
      'reason': 'The letter 'R' is in the correct position and the letters 'A', 'E' are in the word but in the wrong position. And 'C', 'T' are not in the word.\
       A good next guess could be "DREAM", which avoids the lower letters and tries the 'A' and 'E' in different spots along with some new consonants.',
      'guess': 'DREAM'
    }}
    </Example>

    Game Status:
    {game_status}

    Output JSON:
    {{
      'reason': 'Reason for the guess'
      'guess': 'Next best word to guess'
    }}
    """
)

guess_chain = guess_prompt | struct_llm

In [ ]:
game_status = 'Game Start!'

guess = guess_chain.invoke({"game_status": game_status})

In [ ]:
guess

### Wordle Solver

In [ ]:
def solve_wordle(word):
  """Solves a Wordle game using an LLM.

  Args:
    word: A Wordle game answer.

  Returns:
    The solution word, or None if the game could not be solved.
    The number of tries it took to solve the game.
  """
  game = wordle.Wordle(word, real_words=True)

  game_status = "Game Start!"

  tries = 0

  while tries < 6:
    guess = guess_chain.invoke({"game_status": game_status})
    print(guess)
    guess = guess.guess.replace(" ", "") # deletes spacings in the output
    response = game.send_guess(guess)

    if isinstance(response, tuple):
      if response[1]:
        print(response)
        tries += 1
        print("Wordle solved! The word was", game.word)
        return guess, tries
      else:
        print(response)
        tries += 1
        game_status += '\n' + 'Result ' + str(tries) + ': ' + response[0]
        if tries == 6:
          print("Failed to solve Wordle within 6 tries.")
          return None, tries
        continue
    else:
      print(response)
      game_status += '\n' + guess + ": "+ response
      continue
# Test Function
word = "EAGLE"
print(solve_wordle(word))

In [ ]:
test_word_list_20 = ['Crisp', 'Blown', 'Heart', 'Gloom', 'Flare', 'Juicy', 'Vapid', 'Mirth', 'Pluck', 'Grave',
                  'Snipe', 'Batch', 'Drown', 'Lofty', 'Quake', 'Shark', 'Moped', 'Whirl', 'Grasp', 'Fjord']
test_word_list_10 = ["Trend", "Brisk", "Giant", "Mouth", "Flick", "Spear", "Dwell", "Grimy", "Vouch", "Knead"]

In [ ]:
def test_wordle_solver(test_word_list):
  """Tests the Wordle solver with a list of words and calculates success rate and average solve tries.

  Args:
    test_word_list: A list of words to test the solver with.

  Returns:
    A tuple containing the success rate and average solve tries.
  """
  success_count = 0
  total_tries = 0

  for word in test_word_list:
    solution, tries = solve_wordle(word)
    if solution:
      success_count += 1
      total_tries += tries

  if success_count > 0:
    success_rate = (success_count / len(test_word_list)) * 100
    average_tries = total_tries / success_count
  else:
    success_rate = 0
    average_tries = 0

  return success_rate, average_tries

# Test the Wordle solver with the test_word_list
# success_rate, average_tries = test_wordle_solver(test_word_list_20)
success_rate, average_tries = test_wordle_solver(test_word_list_10)
print(f"Success Rate: {success_rate:.2f}%")
print(f"Average Solve Tries: {average_tries:.2f}")

## TODO: Improve Wordle Solver Agent



### Pre-defined Tools

You can use, modify these pre-defined tools.

You can also define your own tools for the Wordle solver agent.

#### Alphabet Tools

In [ ]:
def find_absent_letters(game_status):
  """
  Finds absent letters in the game_status, excluding the game messages.

  Returns:
    A string list of absent letters.
  """
  absent_letters = set()
  lines = game_status.splitlines()
  for line in lines:
    if line.startswith("Result"):
      parts = line.split(":")
      if len(parts) > 1:
        for char in parts[1]:
          if 'a' <= char <= 'z':
            absent_letters.add(char)
  return sorted(list(absent_letters))

In [ ]:
game_status = """Game Start!
Result 1: c   r   A   n   *E*
Result 2: A   n   *G*   *L*   *E*
"""
result = find_absent_letters(game_status)
print(result)

In [ ]:
def find_possible_letters(game_status):
  """
  Finds possible letters in the game_status, excluding the game messages.
  Possible letters are the letters that are not in the absent letters.

  Returns:
    A string list of possible letters.
  """
  absent_letters = set()
  present_letters = set()
  lines = game_status.splitlines()
  for line in lines:
    if line.startswith("Result"):
      parts = line.split(":")
      if len(parts) > 1:
        for char in parts[1]:
          if 'a' <= char <= 'z':
            absent_letters.add(char)

  all_letters = set("abcdefghijklmnopqrstuvwxyz")
  possible_letters = sorted(list(all_letters - absent_letters))
  return possible_letters

In [ ]:
game_status = """Game Start!
Result 1: c   r   A   n   *E*
Result 2: A   n   *G*   *L*   *E*
"""
result = find_possible_letters(game_status)
print(result)

In [ ]:
','.join(result)

### Guess Prompt

In [ ]:
from langchain_core.prompts import PromptTemplate
guess_prompt = PromptTemplate(
    input_variables= ####### TODO #######
    ,
    template="""
    ####### TODO #######
    """
)

### Class for Structured Output

In [ ]:
from typing import List
from pydantic import BaseModel, Field

# TypedDict
class
####### TODO #######

In [ ]:
# Prepare model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.00, api_key=OPENAI_API_KEY, max_tokens=512)
struct_llm = llm.with_structured_output(Guess)

In [ ]:
guess_chain = guess_prompt | struct_llm

#### Wordle Solver

In [ ]:
def solve_wordle(word):
  """Solves a Wordle game using an LLM.

  Args:
    word: A Wordle game answer.

  Returns:
    The solution word, or None if the game could not be solved.
    The number of tries it took to solve the game.
  """
  game = wordle.Wordle(word, real_words=True)

  game_status = "Game Start!"
  previous_guesses = []

  tries = 0
  patience = 9

  while tries < 6:
    absent_letters = find_absent_letters(game_status)
    guess = guess_chain.invoke({####### TODO: fill in with input variables for your own prompt template #######})
    print(guess)
    try:
      guess = guess.guess.replace(" ", "") # deletes spacings in the output
    except AttributeError:
      print("Failed to extract guess from response:", guess)
      guess = "CRANE"
    response = game.send_guess(guess)
    previous_guesses.append(guess)


    if isinstance(response, tuple):
      if response[1]:
        print(response)
        tries += 1
        print("Wordle solved! The word was", game.word)
        return guess, tries
      else:
        print(response)
        tries += 1
        game_status += '\n' + 'Result ' + str(tries) + ': ' + response[0]
        if tries == 6:
          print("Failed to solve Wordle within 6 tries.")
          return None, tries
        continue
    else:
      print(response)
      patience -= 1
      if patience < 0:
        print("Failed to solve because of infinite loop.")
        return None, tries
      game_status += '\n' + guess + ": "+ response
      continue

In [ ]:
# Test Function
word = "Eagle"
print(solve_wordle(word))

In [ ]:
test_word_list_20 = ['Crisp', 'Blown', 'Heart', 'Gloom', 'Flare', 'Juicy', 'Vapid', 'Mirth', 'Pluck', 'Grave',
                  'Snipe', 'Batch', 'Drown', 'Lofty', 'Quake', 'Shark', 'Moped', 'Whirl', 'Grasp', 'Fjord']
test_word_list_10 = ["Trend", "Brisk", "Giant", "Mouth", "Flick", "Spear", "Dwell", "Grimy", "Vouch", "Knead"]

In [ ]:
def test_wordle_solver(test_word_list):
  """Tests the Wordle solver with a list of words and calculates success rate and average solve tries.

  Args:
    test_word_list: A list of words to test the solver with.

  Returns:
    A tuple containing the success rate and average solve tries.
  """
  success_count = 0
  total_tries = 0

  for word in test_word_list:
    print("Given word: " + word)
    solution, tries = solve_wordle(word)
    if solution:
      success_count += 1
      total_tries += tries
    print("-------------------------")

  if success_count > 0:
    success_rate = (success_count / len(test_word_list)) * 100
    average_tries = total_tries / success_count
  else:
    success_rate = 0
    average_tries = 0

  return success_rate, average_tries

# Test the Wordle solver with the test_word_list
success_rate, average_tries = test_wordle_solver(test_word_list_20)
# success_rate, average_tries = test_wordle_solver(test_word_list_10)
print(f"Success Rate: {success_rate:.2f}%")
print(f"Average Solve Tries: {average_tries:.2f}")